In [4]:
import numpy as np
import pandas as pd

#### Homework: Evaluation and Monitoring
In this homework, we'll evaluate the quality of our RAG system.
> It's possible that your answers won't match exactly. If it's the case, select the closest one.
Solution:
* Video: TBA
* Notebook: TBA

#### Getting the data

Let's start by getting the dataset. We will use the data we generated in the module.
In particular, we'll evaluate the quality of our RAG system
with [gpt-4o-mini](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv)

Read it:


In [1]:
github_url="https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"
url = f'{github_url}?raw=1'
url

'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv?raw=1'

In [6]:
df = pd.read_csv(url)
#We will use only the first 300 documents:
df = df.iloc[:300]
# df


#### Q1. Getting the embeddings model

Now, get the embeddings model `multi-qa-mpnet-base-dot-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

> Note: this is not the same model as in HW3


In [8]:


from sentence_transformers import SentenceTransformer

model_name="multi-qa-mpnet-base-dot-v1"
embedding_model = SentenceTransformer(model_name)



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Create the embeddings for the first LLM answer:


In [9]:
answer_llm = df.iloc[0].answer_llm
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [10]:
embedding_model.encode(answer_llm)

array([-4.22446728e-01, -2.24856287e-01, -3.24058354e-01, -2.84758508e-01,
        7.25652836e-03,  1.01186298e-01,  1.03716642e-01, -1.89983487e-01,
       -2.80596633e-02,  2.71589100e-01, -1.15337454e-01,  1.14666104e-01,
       -8.49585757e-02,  3.32365394e-01,  5.52723408e-02, -2.22195745e-01,
       -1.42540783e-01,  1.02519117e-01, -1.52333945e-01, -2.02912569e-01,
        1.98424459e-02,  8.38147402e-02, -5.68632185e-01,  2.32843198e-02,
       -1.67292699e-01, -2.39256889e-01, -8.05465952e-02,  2.57082619e-02,
       -8.15464780e-02, -7.39288479e-02, -2.61550009e-01,  1.92576423e-02,
        3.22909385e-01,  1.90357149e-01, -9.34726413e-05, -2.13165641e-01,
        2.88940948e-02, -1.79529414e-02, -5.92758060e-02,  1.99918360e-01,
       -4.75171655e-02,  1.71634123e-01, -2.45916974e-02, -9.38061401e-02,
       -3.57002705e-01,  1.33263841e-01,  1.94045827e-01, -1.18530430e-01,
        4.56915319e-01,  1.47728100e-01,  3.35945159e-01, -1.86959296e-01,
        2.45954767e-01, -


What's the first value of the resulting vector?

* -0.42  (o)
* -0.22
* -0.02
* 0.21




## Q2. Computing the dot product


Now for each answer pair, let's create embeddings and compute dot product between them

We will put the results (scores) into the `evaluations` list

What's the 75% percentile of the score?

* 21.67
* 31.67 (o)
* 41.67
* 51.67


In [13]:
# a=df.iterrows()
# next(a)

(3,
 answer_llm     The context does not provide any specific info...
 answer_orig    Machine Learning Zoomcamp FAQ\nThe purpose of ...
 document                                                0227b872
 question       Does this course have a GitHub repository for ...
 course                                 machine-learning-zoomcamp
 Name: 3, dtype: object)

In [19]:
import numpy as np

evaluations = []

for _, row in df.iterrows():
    # 為 LLM 答案和人工答案創建嵌入
    embedding_llm = embedding_model.encode(row.answer_llm)
    embedding_human = embedding_model.encode(row.answer_orig)
    
    # 計算點積
    score = np.dot(embedding_llm, embedding_human)
    
    # 將結果添加到評估列表中
    evaluations.append(score)

In [20]:
percentile_75 = np.percentile(evaluations, 75)
print(f"The 75th percentile of the score is: {percentile_75}")

The 75th percentile of the score is: 31.674307346343994


In [22]:
# evaluations[1]

13.418402


#### Q3. Computing the cosine

From Q2, we can see that the results are not within the [0, 1] range. It's because the vectors coming from this model are not normalized.

So we need to normalize them.

To do it, we 

* Compute the norm of a vector
* Divide each element by this norm

So, for vector `v`, it'll be `v / ||v||`

In numpy, this is how you do it:

```python
norm = np.sqrt((v * v).sum())
v_norm = v / norm
```

Let's put it into a function and then compute dot product 
between normalized vectors. This will give us cosine similarity

What's the 75% cosine in the scores?

* 0.63
* 0.73
* 0.83  (o)
* 0.93


In [23]:
import numpy as np

def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

In [24]:

cosine_scores = []

for _, row in df.iterrows():
    # 為 LLM 答案和人工答案創建嵌入
    embedding_llm = embedding_model.encode(row.answer_llm)
    embedding_orig = embedding_model.encode(row.answer_orig)
    normalized_llm=normalize_vector(embedding_llm)
    normalized_orig=normalize_vector(embedding_orig)
    
    # 計算點積
    score = np.dot(normalized_llm, normalized_orig)
    
    # 將結果添加到評估列表中
    cosine_scores.append(score)

In [25]:
percentile_75_cosine = np.percentile(cosine_scores, 75)
print(f"The 75th percentile of the cosine similarity scores is: {percentile_75_cosine}")

The 75th percentile of the cosine similarity scores is: 0.8362348228693008


In [26]:
! pip install rouge



#### Q4. Rouge

Now we will explore an alternative metric - the ROUGE score.  

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

We don't need to implement it ourselves, there's a python package for it:

```bash
pip install rouge
```

(The latest version at the moment of writing is `1.0.1`)

Let's compute the ROUGE score between the answers at the index 10 of our dataframe (`doc_id=5170565b`)

```
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
```

There are three scores: `rouge-1`, `rouge-2` and `rouge-l`, and precision, recall and F1 score for each.

* `rouge-1` - the overlap of unigrams,
* `rouge-2` - bigrams,
* `rouge-l` - the longest common subsequence

What's the F score for `rouge-1`?

- 0.35
- 0.45 (o)
- 0.55
- 0.65


In [28]:
from rouge import Rouge
rouge_scorer = Rouge()
r = df.iloc[10]
r

answer_llm     Yes, all sessions are recorded, so if you miss...
answer_orig    Everything is recorded, so you won’t miss anyt...
document                                                5170565b
question                    Are sessions recorded if I miss one?
course                                 machine-learning-zoomcamp
Name: 10, dtype: object

In [29]:

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [30]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [31]:
rouge_1_f1 = scores['rouge-1']['f']
print(f"The F1 score for rouge-1 is: {rouge_1_f1}")

The F1 score for rouge-1 is: 0.45454544954545456



#### Q5. Average rouge score

Let's compute the average F-score between `rouge-1`, `rouge-2` and `rouge-l` for the same record from Q4

- 0.35 (o)
- 0.45
- 0.55
- 0.65



In [32]:

rouge_1_f1 = scores['rouge-1']['f']
rouge_2_f1 = scores['rouge-2']['f']
rouge_l_f1 = scores['rouge-l']['f']

average_f1 = (rouge_1_f1 + rouge_2_f1 + rouge_l_f1) / 3
print(f"The average F1 score between rouge-1, rouge-2, and rouge-l is: {average_f1}")

The average F1 score between rouge-1, rouge-2, and rouge-l is: 0.35490034990035496



#### Q6. Average rouge score for all the data points

Now let's compute the score for all the records and create a dataframe from them.

What's the average `rouge_2` across all the records?

- 0.10
- 0.20 (o)
- 0.30
- 0.40



In [35]:
scores_list = [rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0] for _,r in df.iterrows()] 

In [38]:
scores_df = pd.DataFrame(scores_list)
for _ in scores_df[:1]['rouge-2']:
    print(_)



{'r': 0.017543859649122806, 'p': 0.07142857142857142, 'f': 0.028169010918468917}


In [39]:
average_rouge_2 = scores_df['rouge-2'].apply(lambda x: x['f']).mean()

print(f"The average rouge-2 score across all records is: {average_rouge_2}")

The average rouge-2 score across all records is: 0.20696501983423318




#### Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw4
* It's possible that your answers won't match exactly. If it's the case, select the closest one.